In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import warnings
warnings.filterwarnings('ignore')

In [2]:
paths = {}
for root, dirs, files in os.walk('.\RawData'):
    if root == '.\RawData': continue
    path_list = []
    for file in files: path_list.append(os.path.join(root, file))
    paths[os.path.split(root)[1]] = path_list
for key in paths.keys():
    print(f"{len(paths[key])} csv files in {key}")

1 csv files in Raw Age Range Data
1 csv files in Raw Covid Data
1 csv files in Raw Income Data
1 csv files in Raw Internet Data
51 csv files in Raw Police Data
1 csv files in Raw Political Affiliation Data
2 csv files in Raw Population Data
1 csv files in Raw Race Breakdown
1 csv files in Raw Violent Crime Data


## Raw Violent Crime

In [3]:
code, name = [], []
state_fip = {}
with open('fip-look-up.txt', 'r') as file:
    file = file.read().split('\n')
    for line in file:
        line = [word for word in line.split(' ') if word != '']
        code.append(int(line[0]))
        if line[-1].upper() in ["COUNTY", "PARISH", "BOROUGH"]: name.append(' '.join(line[1:-1]).upper())
        else: name.append(' '.join(line[1:]).upper())
        if int(line[0]) % 1000 == 0:
            state_fip[' '.join(line[1:]).upper()] = int(int(line[0]) / 1000)
lookup = pd.DataFrame({"code": code, "name": name})
lookup

,code,name
0,1000,ALABAMA
1,1001,AUTAUGA
2,1003,BALDWIN
3,1005,BARBOUR
4,1007,BIBB
...,...,...
3190,56037,SWEETWATER
3191,56039,TETON
3192,56041,UINTA
3193,56043,WASHAKIE


In [4]:
def look_up_fip(state, county):
    try: 
        fip_ = state_fip[state.upper()] * 1000
        county_fip = lookup[(lookup['code'] >= fip_) & (lookup['code'] <= fip_ + 1000) & (lookup['name'] == county.upper())]['code']
        county_fip = county_fip.iloc[0]
    except (IndexError, KeyError): county_fip = -1
    return county_fip

look_up_fip('Alaska', 'Yakutat')

2282

In [5]:
df_all = pd.DataFrame()
violent_df = pd.read_csv(paths['Raw Violent Crime Data'][0])
violent_df.columns = ['County',	'Violent crime', 'Murder and nonnegligent manslaughter','Rape1', 'Rape2', 'Robbery', 'Aggravated assault', 'Property crime', 'Burglary',	'Larceny-theft', 'Motor vehicle theft', 'Arson3', 'State']
nul = violent_df['State'].isna()
exceptions = ["NEW", "NORTH", "SOUTH", "WEST"]
for i in range(len(violent_df)):
    if nul[i] == True: violent_df.loc[i, 'State'] = violent_df.loc[i-1, 'State']
    tmp = violent_df.loc[i, 'State'].split(' ')[0]
    if tmp in exceptions: violent_df.loc[i, 'State'] = " ".join(violent_df.loc[i, 'State'].split(' ')[0:2])
    else: violent_df.loc[i, 'State'] = tmp
df_all = pd.concat([df_all, violent_df], axis = 1)
df_all.loc[df_all.County == 'Baltimore County', 'County'] = 'Baltimore'
violent_df.head(3)

,County,Violent crime,Murder and nonnegligent manslaughter,Rape1,Rape2,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson3,State
0,Autauga,73,1,12.0,NaN,8,52,429,146,233,50,NaN,ALABAMA
1,Baldwin,127,1,5.0,NaN,23,98,613,229,342,42,NaN,ALABAMA
2,Bibb,0,0,0.0,NaN,0,0,37,20,14,3,NaN,ALABAMA


In [6]:
violent_df['State'].unique()

array(['ALABAMA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO',
       'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS',
       'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE',
       'MARYLAND', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI',
       'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY',
       'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO',
       'OKLAHOMA', 'OREGON', 'PENNSYLVANIA', 'SOUTH CAROLINA',
       'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT',
       'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING'],
      dtype=object)

In [7]:
# Now edit out government agencies and unrecognizable counties
df_all = df_all.set_index(['State', 'County'])
remove = [('CALIFORNIA', 'Tulare4'),
('COLORADO', 'Eagle5'),
('DELAWARE', 'New Castle County Police Department'),
('GEORGIA', 'Clayton County Police Department'),
('GEORGIA', 'Cobb County Police Department'),
('GEORGIA', 'DeKalb County Police Department'),
('GEORGIA', 'Dougherty County Police Department'),
('GEORGIA', 'Floyd County Police Department'),
('GEORGIA', 'Fulton County Police Department'),
('GEORGIA', 'Glynn County Police Department'),
('GEORGIA', 'Gwinnett County Police Department'),
('GEORGIA', 'Henry County Police Department'),
('GEORGIA', 'Polk County Police Department'),
('HAWAII', 'Maui Police Department'),
('HAWAII', 'Hawaii Police Department'),
('HAWAII', 'Kauai Police Department'),
('KANSAS', 'Riley County Police Department'),
('KANSAS', 'Cherokee6'),
('KANSAS', 'Ford6'),
('KANSAS', 'Labette6'),
('KANSAS', 'Pawnee6'),
('KENTUCKY', 'Campbell County Police Department'),
('KENTUCKY', 'Kenton County Police Department'),
('KENTUCKY', 'Oldham County Police Department'),
('KENTUCKY', 'Muhlenberg County Police Department'),
('LOUISIANA', 'Iberia5'),
('LOUISIANA', 'La Salle'),
('MAINE', 'Androscoggin7'),
('MAINE', 'Cumberland7'),
('MAINE', 'Penobscot7'),
('MAINE', 'Sagadahoc7'),
('MAINE', 'York7'),
('MAINE', 'Aroostook7'),
('MAINE', 'Hancock7'),
('MAINE', 'Knox7'),
('MAINE', 'Lincoln7'),
('MAINE', 'Oxford7'),
('MAINE', 'Piscataquis7'),
('MAINE', 'Waldo7'),
('MAINE', 'Washington7'),
('MARYLAND', 'Anne Arundel County Police Department'),
('MARYLAND', 'Baltimore County Police Department'),
('MARYLAND', 'Howard County Police Department'),
('MARYLAND', 'Montgomery County Police Department'),
('MARYLAND', "Prince George's "),
('MINNESOTA', 'Carver8'),
('MISSISSIPPI', 'Hinds8 '),
('MISSOURI', 'Newton9'),
('MISSOURI', 'St. Charles County Police Department'),
('MISSOURI', 'St. Louis County Police Department'),
('MISSOURI', 'Camden5'),
('NEW JERSEY', 'Camden County Police Department'),
('NEW YORK', 'Suffolk County Police Department'),
('NEW YORK', 'Westchester Public Safety'),
('NEW YORK', 'Fulton7'),
('NORTH CAROLINA', 'Halifax5'),
('NORTH CAROLINA', 'Richmond5'),
('NORTH CAROLINA', 'Sampson5'),
('NORTH CAROLINA', 'Scotland5'),
('SOUTH CAROLINA', 'Horry County Police Department'),
('SOUTH CAROLINA', 'Marion9'),
('TEXAS', 'Bosque9'),
('TEXAS', 'Lavaca9'),
('TEXAS', 'Trinity9'),
('UTAH', 'Salt Lake County Unified Police Department'),
('VIRGINIA', 'Albemarle County Police Department'),
('VIRGINIA', 'Arlington County Police Department'),
('VIRGINIA', 'Chesterfield County Police Department'),
('VIRGINIA', 'Fairfax County Police Department'),
('VIRGINIA', 'Henrico County Police Department'),
('VIRGINIA', 'James City County Police Department'),
('VIRGINIA', 'Prince George County Police Department'),
('VIRGINIA', 'Prince William County Police Department'),
('VIRGINIA', 'Roanoke County Police Department'),
('WASHINGTON', 'Spokane8'),
('WISCONSIN', 'Brown8'),
('WISCONSIN', 'Waushara8')
]
df_all.drop([index for index in remove], axis = 0, inplace = True)
df_all

Violent crime  Murder and nonnegligent manslaughter  Rape1  \
State   County                                                                  
ALABAMA Autauga               73                                     1   12.0   
        Baldwin              127                                     1    5.0   
        Bibb                   0                                     0    0.0   
        Blount               394                                     1   17.0   
        Calhoun               23                                     0    7.0   
...                          ...                                   ...    ...   
WYOMING Sublette               3                                     0    0.0   
        Sweetwater            25                                     0    7.0   
        Uinta                  4                                     0    0.0   
        Washakie               0                                     0    0.0   
        Weston                 1                                     0    0.0   

                    Rape2 Robbery Aggravated assault Property crime Burglary  \
State   County                                                                 
ALABAMA Autauga       NaN       8                 52            429      146   
        Baldwin       NaN      23                 98            613      229   
        Bibb          NaN       0                  0             37       20   
        Blount        NaN       9                367            867      261   
        Calhoun       NaN       5                 11            319      137   
...                   ...     ...                ...            ...      ...   
WYOMING Sublette      NaN       1                  2             79        5   
        Sweetwater    NaN       0                 18            124       33   
        Uinta         NaN       0                  4             60        5   
        Washakie      NaN       0                  0             14        4   
        Weston        NaN       0                  1              3        0   

                   Larceny-theft Motor vehicle theft  Arson3  
State   County                                                
ALABAMA Autauga              233                  50     NaN  
        Baldwin              342                  42     NaN  
        Bibb                  14                   3     NaN  
        Blount               501                 105     NaN  
        Calhoun              181                   1     NaN  
...                          ...                 ...     ...  
WYOMING Sublette              69                   5     0.0  
        Sweetwater            69                  22     5.0  
        Uinta                 50                   5     0.0  
        Washakie               7                   3     0.0  
        Weston                 3                   0     0.0  

[2535 rows x 11 columns]

In [8]:
# Use fip as index
def convert(df):
    new_idxs = [look_up_fip(state, county) for (state, county) in df.index.values]
    df.index = new_idxs 
    df.drop([-1], inplace = True, axis = 0)
    df[df.index == -1]

convert(df_all)

## Raw Age Range Data

In [9]:
age_df = pd.read_csv(paths['Raw Age Range Data'][0])
age_df.head(5)

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,...,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
0,50,1,1,Alabama,Autauga County,1,58802,28761,30041,3443,...,1055,1165,501,664,955,345,610,39.2,37.9,40.3
1,50,1,1,Alabama,Autauga County,2,58902,28819,30083,3457,...,1064,1170,501,669,958,350,608,39.1,37.9,40.3
2,50,1,1,Alabama,Autauga County,3,59210,28851,30359,3416,...,1074,1190,492,698,972,362,610,39.2,38.0,40.3
3,50,1,1,Alabama,Autauga County,4,59759,29105,30654,3460,...,1152,1225,495,730,1019,385,634,39.2,38.0,40.4
4,50,1,3,Alabama,Baldwin County,1,231761,113388,118373,12046,...,4861,5601,2542,3059,4610,1812,2798,43.7,42.5,44.9


In [10]:
age_df.rename(columns = {'STNAME': 'State', "CTYNAME": 'County'}, inplace = True)
end_words = ['County', 'Borough', 'Parish']
age_df['County'] = age_df['County'].apply(lambda x: ' '.join(x.split(' ')[0:-1]) if x.split(' ')[-1] in end_words else x)
age_df['State'] = age_df['State'].apply(lambda x: x.upper())
age_df.set_index(['State', "County"], inplace=True)
age_df.sample(10)

,,SUMLEV,STATE,COUNTY,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,UNDER5_MALE,UNDER5_FEM,...,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
State,County,,,,,,,,,,,,,,,,,,,,,
LOUISIANA,Lafourche,50,22,57,1,97552,48296,49256,5804,2913,2891,...,1551,1872,804,1068,1772,591,1181,38.4,37.7,39.1
MISSOURI,Clinton,50,29,49,2,21210,10712,10498,1205,593,612,...,352,462,220,242,513,210,303,41.5,41.1,41.9
INDIANA,Morgan,50,18,109,4,72236,36121,36115,3729,1894,1835,...,1332,1503,655,848,1184,432,752,42.3,41.5,43.2
VIRGINIA,Loudoun,50,51,107,3,428435,215027,213408,26999,13755,13244,...,4549,5076,2166,2910,5042,1867,3175,37.6,37.4,37.8
ILLINOIS,Moultrie,50,17,139,3,14509,7174,7335,836,439,397,...,233,312,124,188,405,124,281,39.6,38.2,41.0
GEORGIA,Polk,50,13,233,1,42854,21178,21676,2817,1417,1400,...,709,746,325,421,607,183,424,37.1,36.0,38.2
CALIFORNIA,San Francisco,50,6,75,3,811253,417449,393804,34332,17464,16868,...,12803,17192,7250,9942,22169,8108,14061,40.5,40.2,40.9
TENNESSEE,Houston,50,47,83,3,8282,4086,4196,424,212,212,...,158,212,94,118,155,45,110,43.4,42.6,44.2
MISSOURI,Madison,50,29,123,3,12671,6363,6308,680,325,355,...,240,283,120,163,294,126,168,42.1,40.8,43.2


In [11]:
age_df = age_df[age_df['YEAR']==4]
age_df.drop(['STATE', 'COUNTY', 'SUMLEV', 'YEAR'], axis = 1, inplace=True)
print(len([idx for idx in age_df.index if idx in df_all.index]))
print(len(df_all.index))

0
2525


In [12]:
convert(age_df)

In [13]:
print("These counties fail to match")
for idx in df_all.index:
    if idx not in age_df.index:
        print(idx)

These counties fail to match
17099
18091
18033
35013


In [14]:
dropidx = [idx for idx in age_df.index if idx not in df_all.index]
age_df.drop(dropidx, axis = 0, inplace = True)
print(len(age_df))
print(len(df_all))
df_all = pd.concat([df_all, age_df], axis = 1)
df_all

2521
2525


,Violent crime,Murder and nonnegligent manslaughter,Rape1,Rape2,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,...,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
1001,73,1,12.0,NaN,8,52,429,146,233,50,...,1152.0,1225.0,495.0,730.0,1019.0,385.0,634.0,39.2,38.0,40.4
1003,127,1,5.0,NaN,23,98,613,229,342,42,...,5709.0,6113.0,2811.0,3302.0,4933.0,1967.0,2966.0,44.1,42.9,45.3
1007,0,0,0.0,NaN,0,0,37,20,14,3,...,460.0,508.0,229.0,279.0,418.0,122.0,296.0,40.9,39.1,43.6
1009,394,1,17.0,NaN,9,367,867,261,501,105,...,1307.0,1328.0,563.0,765.0,1222.0,467.0,755.0,41.2,40.4,42.1
1015,23,0,7.0,NaN,5,11,319,137,181,1,...,2322.0,2431.0,997.0,1434.0,2101.0,595.0,1506.0,39.3,37.6,40.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,3,0,0.0,NaN,1,2,79,5,69,5,...,206.0,237.0,119.0,118.0,184.0,99.0,85.0,44.7,44.8,44.7
56037,25,0,7.0,NaN,0,18,124,33,69,22,...,503.0,590.0,287.0,303.0,498.0,174.0,324.0,37.7,37.8,37.6
56041,4,0,0.0,NaN,0,4,60,5,50,5,...,255.0,316.0,147.0,169.0,277.0,104.0,173.0,38.0,38.1,37.9
56043,0,0,0.0,NaN,0,0,14,4,7,3,...,187.0,250.0,125.0,125.0,212.0,73.0,139.0,44.8,43.3,46.6


## COVID

In [15]:
ref = pd.read_csv('state-ref.csv')
code_to_name = {code:name.upper() for (code, name) in zip(ref['code'].values, ref['state'].values)}
code_to_name['US'] = 'US'
code_to_name['PR'] = 'PR'
code_to_name['HAWAII'] = 'HAWAII'
code_to_name['HI'] = 'HAWAII'

In [16]:
covid_path = paths['Raw Covid Data'][0]
covid_df = pd.read_csv(covid_path)
covid_df['State'] = covid_df['State'].apply(lambda x: code_to_name[x])
covid_df = covid_df.set_index(['State', 'County'])
cols = ["PerCapitaInc", "PovertyAllAgesPct", "Immigration_Rate_2000_2010", "Ed1LessThanHSPct", "Ed3SomeCollegePct", "Ed5CollegePlusPct", "OwnHomePct", "NumUnemployed2019"]
idxs = [idx for idx in covid_df.index if idx[0] not in ['US', 'PR']]
covid_df = covid_df.loc[idxs, cols]
covid_df.sample(3)

,,PerCapitaInc,PovertyAllAgesPct,Immigration_Rate_2000_2010,Ed1LessThanHSPct,Ed3SomeCollegePct,Ed5CollegePlusPct,OwnHomePct,NumUnemployed2019
State,County,,,,,,,,
PENNSYLVANIA,Susquehanna,29630.0,12.8,0.002366,10.812316,17.242057,17.841467,76.578083,889.0
MISSISSIPPI,Lafayette,26154.0,18.0,1.688640,10.536374,19.106692,43.926929,61.199979,1280.0
INDIANA,Floyd,32299.0,9.5,0.368939,9.722460,21.572661,29.544545,72.438343,1321.0


In [17]:
convert(covid_df)
covid_df.sample(3)

,PerCapitaInc,PovertyAllAgesPct,Immigration_Rate_2000_2010,Ed1LessThanHSPct,Ed3SomeCollegePct,Ed5CollegePlusPct,OwnHomePct,NumUnemployed2019
54051,24882.0,13.9,0.097460,9.042093,20.989953,18.504244,80.264879,902.0
37131,20538.0,21.4,0.147452,20.546943,17.366425,12.808570,69.042652,406.0
13091,18921.0,26.5,0.834725,16.760450,19.239014,13.699089,66.168443,381.0


In [18]:
vis = {idx : True for idx in covid_df.index}
discard = []
for idx in covid_df.index:
    if vis[idx] == False:
        discard.append(idx)
    vis[idx] = False
discard = np.unique(discard)
covid_df.drop(discard, inplace = True, axis = 0)

In [19]:
idxs = [idx for idx in covid_df.index if idx in df_all.index]
covid_df = covid_df.loc[idxs, cols]
df_all = pd.concat([df_all, covid_df], axis = 1)
df_all.sample(3)

,Violent crime,Murder and nonnegligent manslaughter,Rape1,Rape2,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,...,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM,PerCapitaInc,PovertyAllAgesPct,Immigration_Rate_2000_2010,Ed1LessThanHSPct,Ed3SomeCollegePct,Ed5CollegePlusPct,OwnHomePct,NumUnemployed2019
37127,70,2,NaN,11.0,5,52,425,183,218,24,...,40.0,43.3,26980.0,14.2,1.081007,14.599477,20.750777,20.888435,65.669935,2099.0
13275,63,0,NaN,7.0,12,44,484,147,289,48,...,38.9,41.8,25542.0,20.2,0.823040,17.383589,21.780421,23.071867,63.180037,663.0
48257,136,1,28.0,NaN,11,96,551,196,275,80,...,33.3,34.4,27411.0,11.5,2.218071,13.830805,24.204565,20.712123,76.300717,2112.0


## Income

This data is redundant with COVID data. 

## Internet

In [20]:
path = paths['Raw Internet Data'][0]
internet_df = pd.read_excel(path)
internet_df.sample(3)

,Year,Month,FIPS,State,County,State_Name,County_Name,Housing_Units,Tier_1,Tier_2,Tier_3,Tier_4
18402,2011,6,41045,41,45,Oregon,Malheur County,11690,3,3,2,1
8019,2012,12,29129,29,129,Missouri,Mercer County,2134,3,3,2,1
35270,2008,12,51595,51,595,Virginia,Emporia city,2552,1,1,1,1


In [21]:
internet_df = internet_df.loc[(internet_df['Year'] == 2013) & (internet_df['Month'] == 12), :]
end_words = ['County', 'Borough', 'Parish']
internet_df['County_Name'] = internet_df['County_Name'].apply(lambda x: ' '.join(x.split(' ')[0:-1]) if x.split(' ')[-1] in end_words else x)
internet_df.set_index(['State_Name', 'County_Name'], inplace = True)
convert(internet_df)

In [22]:
idxs = [idx for idx in internet_df.index if idx in df_all.index]
internet_df = internet_df.loc[idxs, :]
df_all = pd.concat([df_all, internet_df], axis = 1)
df_all.sample(3)

,Violent crime,Murder and nonnegligent manslaughter,Rape1,Rape2,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,...,Year,Month,FIPS,State,County,Housing_Units,Tier_1,Tier_2,Tier_3,Tier_4
20173,24,4,1.0,NaN,1,18,89,37,31,21,...,2013.0,12.0,20173.0,20.0,173.0,214093.0,4.0,4.0,3.0,2.0
38011,0,0,0.0,NaN,0,0,7,1,1,5,...,2013.0,12.0,38011.0,38.0,11.0,1726.0,4.0,4.0,3.0,3.0
39141,120,1,31.0,NaN,12,76,"1,316",405,844,67,...,2013.0,12.0,39141.0,39.0,141.0,32083.0,4.0,4.0,3.0,2.0


## Raw Police Data

In [23]:
police_by_state = {}
for path in paths['Raw Police Data']:
    state_abbr = os.path.basename(path)[0:2]
    state = code_to_name[state_abbr]
    police_by_state[state] = pd.read_csv(path)

In [24]:
df_police = pd.DataFrame()
cols = ['male_officer', 'female_officer', "county_name"]
for (state, df_police_) in police_by_state.items():
    county_names = df_police_['county_name'].apply(lambda x: x.split(',')[0]).unique()
    df_police_ = df_police_.loc[:, cols]
    df_police_ = df_police_.groupby('county_name').sum() # by county sum
    df_police_['state'] = [state] * len(df_police_)
    df_police_['county'] = df_police_.index
    df_police_.set_index(['state', 'county'], inplace = True)
    df_police = pd.concat([df_police, df_police_], axis = 0)

In [25]:
convert(df_police)
df_police

,male_officer,female_officer
2060,4.0,0.0
2090,46.0,3.0
2100,4.0,0.0
2122,49.0,2.0
2130,22.0,1.0
...,...,...
56037,95.0,34.0
56039,27.0,4.0
56041,46.0,3.0
56043,18.0,1.0


In [26]:
idxs = [idx for idx in df_police.index if idx in df_all.index]
df_police = df_police.loc[idxs, :]
df_all = pd.concat([df_all, df_police], axis = 1)
df_all.sample(3)

,Violent crime,Murder and nonnegligent manslaughter,Rape1,Rape2,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,...,FIPS,State,County,Housing_Units,Tier_1,Tier_2,Tier_3,Tier_4,male_officer,female_officer
37101,160,3,NaN,16.0,39,102,"1,874",776,"1,000",98,...,37101.0,37.0,101.0,69566.0,4.0,4.0,4.0,3.0,235.0,30.0
20169,25,1,4.0,NaN,0,20,128,39,74,15,...,20169.0,20.0,169.0,24264.0,3.0,3.0,3.0,2.0,100.0,15.0
51117,32,2,11.0,NaN,4,15,219,84,122,13,...,51117.0,51.0,117.0,18786.0,2.0,2.0,1.0,1.0,81.0,5.0


## Party Affiliation

In [27]:
path = paths['Raw Political Affiliation Data'][0]
df_affiliation = pd.read_excel(path)
for i in range(len(df_affiliation)):
    try:
        df_affiliation.loc[i, 'State'] = lookup.loc[(lookup['name']==df_affiliation.loc[i, 'State'].upper()) & (lookup['code'] % 1000 == 0), 'code'].iloc[0]
    except KeyError:
        df_affiliation.drop(i, axis = 0, inplace=True)
df_affiliation.set_index('State', inplace=True)
df_affiliation

,Republican/lean Rep.,No lean,Democrat/lean Dem.,Sample size
State,,,,
1000,0.52,0.13,0.35,511
2000,0.39,0.29,0.32,310
4000,0.40,0.21,0.39,653
5000,0.46,0.16,0.38,311
6000,0.30,0.21,0.49,3697
8000,0.41,0.17,0.42,504
9000,0.32,0.18,0.50,377
10000,0.29,0.17,0.55,301
11000,0.11,0.15,0.73,303


In [28]:
df_all = pd.concat([df_all, pd.DataFrame({"Republican":[], "Democrat":[]})])
for i, idx in enumerate(df_all.index):
    state_fip_ = int(idx / 1000) * 1000
    df_all.loc[idx, "Republican"] = df_affiliation.loc[state_fip_, "Republican/lean Rep."]
    df_all.loc[idx, "Democrat"] = df_affiliation.loc[state_fip_, "Democrat/lean Dem."]
df_all.sample(3)

,Violent crime,Murder and nonnegligent manslaughter,Rape1,Rape2,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,...,County,Housing_Units,Tier_1,Tier_2,Tier_3,Tier_4,male_officer,female_officer,Republican,Democrat
17091,45,5.0,1.0,NaN,6,33,366,158,185,23,...,91.0,45304.0,4.0,4.0,3.0,3.0,NaN,NaN,0.33,0.48
18177,12,1.0,5.0,NaN,1,5,275,104,162,9,...,177.0,31276.0,3.0,3.0,3.0,2.0,0.0,0.0,0.42,0.37
47063,111,0.0,7.0,NaN,5,99,458,148,251,59,...,63.0,27057.0,2.0,2.0,2.0,2.0,101.0,9.0,0.48,0.36


## Population

In [29]:
path = paths['Raw Population Data'][0]
population_df = pd.read_csv(path)
population_df.drop(0, axis = 0, inplace=True)
population_df.head(3)

,Region,2020_,2020,2021,2022
1,".Autauga County, Alabama","58,802","58,902","59,210","59,759"
2,".Baldwin County, Alabama","231,761","233,219","239,361","246,435"
3,".Barbour County, Alabama","25,224","24,960","24,539","24,706"


In [30]:
population_df['County'] = population_df['Region'].apply(lambda x: x.split(',')[0][1:])
population_df['State'] = population_df['Region'].apply(lambda x: x.split(',')[1][1:])
end_words = ['County', 'Borough', 'Parish']
population_df['County'] = population_df['County'].apply(lambda x: ' '.join(x.split(' ')[0:-1]) if x.split(' ')[-1] in end_words else x)
population_df.set_index(['State', 'County'], inplace = True)
convert(population_df)
population_df.drop(['Region', '2020_', '2021', '2020'], axis = 1, inplace = True)
population_df

,2022
1001,"59,759"
1003,"246,435"
1005,"24,706"
1007,"22,005"
1009,"59,512"
...,...
56037,"41,345"
56039,"23,287"
56041,"20,712"
56043,"7,719"


In [31]:
idxs = [idx for idx in population_df.index if idx in df_all.index]
population_df = population_df.loc[idxs, :]
df_all = pd.concat([df_all, population_df], axis = 1)
df_all

,Violent crime,Murder and nonnegligent manslaughter,Rape1,Rape2,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,...,Housing_Units,Tier_1,Tier_2,Tier_3,Tier_4,male_officer,female_officer,Republican,Democrat,2022
1001,73,1.0,12.0,NaN,8,52,429,146,233,50,...,22820.0,4.0,4.0,3.0,3.0,35.0,3.0,0.52,0.35,"59,759"
1003,127,1.0,5.0,NaN,23,98,613,229,342,42,...,106739.0,3.0,3.0,3.0,2.0,346.0,29.0,0.52,0.35,"246,435"
1007,0,0.0,0.0,NaN,0,0,37,20,14,3,...,9048.0,2.0,2.0,2.0,1.0,24.0,0.0,0.52,0.35,"22,005"
1009,394,1.0,17.0,NaN,9,367,867,261,501,105,...,24020.0,3.0,3.0,2.0,1.0,80.0,7.0,0.52,0.35,"59,512"
1015,23,0.0,7.0,NaN,5,11,319,137,181,1,...,53359.0,3.0,3.0,3.0,2.0,173.0,13.0,0.52,0.35,"115,788"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,3,0.0,0.0,NaN,1,2,79,5,69,5,...,5876.0,3.0,3.0,2.0,1.0,40.0,11.0,0.57,0.25,"8,763"
56037,25,0.0,7.0,NaN,0,18,124,33,69,22,...,19249.0,4.0,4.0,3.0,2.0,95.0,34.0,0.57,0.25,"41,345"
56041,4,0.0,0.0,NaN,0,4,60,5,50,5,...,8864.0,3.0,3.0,2.0,1.0,46.0,3.0,0.57,0.25,"20,712"
56043,0,0.0,0.0,NaN,0,0,14,4,7,3,...,3849.0,3.0,3.0,3.0,2.0,18.0,1.0,0.57,0.25,"7,719"


## Race

The formula for diversity index is

$$D = 1 - \sum_{n \in S} \frac{n(n-1)}{N(N-1)}$$

where $n$ is the number of individuals of a single species, $S$ is the set of all species, and $N$ is the number of individuals in the total population

In [32]:
path = paths['Raw Race Breakdown'][0]
race_df = pd.read_csv(path)
race_df.sample(3)

,Location,Diversity-Index,"Black or African American alone, percent, 2013","American Indian and Alaska Native alone, percent, 2013","Asian alone, percent, 2013","Native Hawaiian and Other Pacific Islander alone, percent,","Two or More Races, percent, 2013","Hispanic or Latino, percent, 2013","White alone, not Hispanic or Latino, percent, 2013"
2146,"Sheridan County, WY",0.162154,1.2,1.4,0.8,0.1,1.4,4.3,91.4
286,"Bee County, TX",0.553516,8.5,0.8,0.6,0.1,1.1,57.4,33.1
1136,"Thurston County, WA",0.391913,3.3,1.7,5.5,0.9,5.1,8.1,77.1


In [33]:
idxs = [idx for idx in race_df.index if ',' in race_df.loc[idx, 'Location']]
idxs_ = [idx for idx in race_df.index if ',' not in race_df.loc[idx, 'Location']]
race_df.loc[idxs, 'State'] = race_df.loc[idxs, 'Location'].apply(lambda x: code_to_name[x.split(' ')[-1]])
race_df.loc[idxs_, 'State'] = race_df.loc[idxs_, 'Location']
race_df.loc[idxs, 'County'] = race_df.loc[idxs, 'Location'].apply(lambda x: x.split(',')[0])
race_df.loc[idxs_, 'County'] = race_df.loc[idxs_, 'Location']
race_df['County'] = race_df['County'].apply(lambda x: ' '.join(x.split(' ')[0:-1]) if x.split(' ')[-1] in end_words else x)
race_df.set_index(['State', 'County'], inplace = True)
convert(race_df)
race_df.drop('Location', axis = 1, inplace = True)
race_df

,Diversity-Index,"Black or African American alone, percent, 2013","American Indian and Alaska Native alone, percent, 2013","Asian alone, percent, 2013","Native Hawaiian and Other Pacific Islander alone, percent,","Two or More Races, percent, 2013","Hispanic or Latino, percent, 2013","White alone, not Hispanic or Latino, percent, 2013"
2016,0.769346,7.4,13.8,31.1,2.3,4.8,14.6,29.2
36081,0.742224,20.9,1.3,25.2,0.2,2.7,28.0,26.7
15009,0.740757,0.8,0.6,28.8,10.6,23.3,10.7,31.5
6001,0.740399,12.4,1.2,28.2,1.0,5.2,22.7,33.2
2013,0.738867,7.7,21.8,41.4,0.7,3.7,13.5,12.9
...,...,...,...,...,...,...,...,...
29151,0.037540,0.3,0.2,0.1,0.0,0.7,0.6,98.1
54043,0.035585,0.2,0.1,0.1,0.0,0.7,0.6,98.2
21131,0.035581,0.4,0.1,0.2,0.0,0.7,0.5,98.2
31009,0.023784,0.2,0.0,0.0,0.0,0.8,0.2,98.8


In [34]:
vis = {idx : True for idx in race_df.index}
discard = []
for idx in race_df.index:
    if vis[idx] == False:
        discard.append(idx)
    vis[idx] = False
discard = np.unique(discard)
race_df.drop(discard, inplace = True, axis = 0)

In [35]:
idxs = [idx for idx in race_df.index if idx in df_all.index]
race_df = race_df.loc[idxs, :]
df_all = pd.concat([df_all, race_df], axis = 1)
df_all

,Violent crime,Murder and nonnegligent manslaughter,Rape1,Rape2,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,...,Democrat,2022,Diversity-Index,"Black or African American alone, percent, 2013","American Indian and Alaska Native alone, percent, 2013","Asian alone, percent, 2013","Native Hawaiian and Other Pacific Islander alone, percent,","Two or More Races, percent, 2013","Hispanic or Latino, percent, 2013","White alone, not Hispanic or Latino, percent, 2013"
1001,73,1.0,12.0,NaN,8,52,429,146,233,50,...,0.35,"59,759",0.388898,18.4,0.5,1.1,0.1,1.7,2.7,75.9
1003,127,1.0,5.0,NaN,23,98,613,229,342,42,...,0.35,"246,435",0.297942,9.5,0.7,0.9,0.1,1.5,4.6,83.1
1007,0,0.0,0.0,NaN,0,0,37,20,14,3,...,0.35,"22,005",0.394100,22.1,0.4,0.2,0.1,0.9,2.1,74.6
1009,394,1.0,17.0,NaN,9,367,867,261,501,105,...,0.35,"59,512",0.221056,1.8,0.6,0.3,0.1,1.1,8.7,87.8
1015,23,0.0,7.0,NaN,5,11,319,137,181,1,...,0.35,"115,788",0.420387,20.9,0.5,0.8,0.1,1.6,3.5,73.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,3,0.0,0.0,NaN,1,2,79,5,69,5,...,0.25,"8,763",0.218882,1.4,1.1,1.1,0.0,1.4,7.8,88.0
56037,25,0.0,7.0,NaN,0,18,124,33,69,22,...,0.25,"41,345",0.340296,1.7,1.2,0.9,0.2,1.7,15.9,79.6
56041,4,0.0,0.0,NaN,0,4,60,5,50,5,...,0.25,"20,712",0.225552,0.8,1.3,0.4,0.2,1.7,9.1,87.5
56043,0,0.0,0.0,NaN,0,0,14,4,7,3,...,0.25,"7,719",0.296868,0.8,1.7,0.7,0.1,1.7,14.2,82.6


## Save new dataset

In [36]:
df_all.drop('FIPS', axis = 1, inplace = True)
df_all.to_csv('crime_re.csv')